<div  align='center'><img src='https://s3.amazonaws.com/weclouddata/images/logos/wcd_logo_new_2.png' width='15%'></div >
<p style="font-size:30px;text-align:center"><b>Lab Model training</b></p>
<p style="font-size:20px;text-align:center"><b><font color='#F39A54'>Data Engineering Diploma</font></b></p>
<p style="font-size:15px;text-align:center">Content developed by: WeCloudData Academy</p>

In this Lab, we are going to use the ML model we trained previously to make sentiment analysis to conclude the topic of each post, and summarize which topics are the most popular topics. The business reason why we train this model is because we are going to create a predict model to give a post a topic immediately when it was created. In this way, the post will be categorized to the right place for other users.

Before you start this lab, please finish the **[Workshop] Mount Azure Storage to Azure Databricks** first, it is the prerequisite of the lab. 

In the model training process, you are going to read the reviews those come from the `Reviews` files we ingest earlier from the public ` de-yelp-daily` container. 

The `Reviews` files are in parquet format. If you recall our previous job in data factory, they are recieved everyday and stored in a folder (in this example, we call it `Landing`). And in the `Landing` folder, the reviews parquet files are saved in the sub-folders, each sub-folder represents a day when the files were created. 

You can mounte it to '/mnt/deBDProject'

In [ ]:
#Original Statement
#spark._jsc.hadoopConfiguration().set("fs.azure.account.key.wcddestorageexternal.dfs.core.windows.net", "")

In [ ]:

from datetime import datetime
current_date_str = "dt=" + datetime.now().strftime("%Y-%m-%d")
#current_date_str = "dt=2024-10-29"
#path = f"abfss://de-yelp-daily@wcddestorageexternal.dfs.core.windows.net/output/{current_date_str}/"
path = f"/mnt/deBDproject/de-yelp-daily/reviews/{current_date_str}/"
display(dbutils.fs.ls(path))

path,name,size,modificationTime
abfss://de-yelp-daily@wcddestorageexternal.dfs.core.windows.net/output/dt=2024-06-18/03f6e983-c920-498b-9837-14fff0a74b81.parquet,03f6e983-c920-498b-9837-14fff0a74b81.parquet,9350,1718735078000
abfss://de-yelp-daily@wcddestorageexternal.dfs.core.windows.net/output/dt=2024-06-18/0831a920-cb58-4fed-a09e-a9e7b2ea0168.parquet,0831a920-cb58-4fed-a09e-a9e7b2ea0168.parquet,8906,1718734825000
abfss://de-yelp-daily@wcddestorageexternal.dfs.core.windows.net/output/dt=2024-06-18/08b2b53f-5c53-4ded-a393-0d0ef989928d.parquet,08b2b53f-5c53-4ded-a393-0d0ef989928d.parquet,11014,1718735078000
abfss://de-yelp-daily@wcddestorageexternal.dfs.core.windows.net/output/dt=2024-06-18/1561f370-6604-420e-84cc-6c4083bf2e97.parquet,1561f370-6604-420e-84cc-6c4083bf2e97.parquet,8237,1718735078000
abfss://de-yelp-daily@wcddestorageexternal.dfs.core.windows.net/output/dt=2024-06-18/1ae7f45f-73c3-4e79-b464-0fd15e24e38f.parquet,1ae7f45f-73c3-4e79-b464-0fd15e24e38f.parquet,9402,1718734826000
abfss://de-yelp-daily@wcddestorageexternal.dfs.core.windows.net/output/dt=2024-06-18/24c5f19a-f4b3-4917-a953-f0fb824dd9b9.parquet,24c5f19a-f4b3-4917-a953-f0fb824dd9b9.parquet,11559,1718734826000
abfss://de-yelp-daily@wcddestorageexternal.dfs.core.windows.net/output/dt=2024-06-18/3cd9c560-8a69-4a39-88e1-22104310df63.parquet,3cd9c560-8a69-4a39-88e1-22104310df63.parquet,10192,1718735079000
abfss://de-yelp-daily@wcddestorageexternal.dfs.core.windows.net/output/dt=2024-06-18/3e2e3380-136e-449a-b38e-e270928f8590.parquet,3e2e3380-136e-449a-b38e-e270928f8590.parquet,9524,1718734807000
abfss://de-yelp-daily@wcddestorageexternal.dfs.core.windows.net/output/dt=2024-06-18/43413387-73f3-448a-8c68-6550e21f09a1.parquet,43413387-73f3-448a-8c68-6550e21f09a1.parquet,7998,1718735078000
abfss://de-yelp-daily@wcddestorageexternal.dfs.core.windows.net/output/dt=2024-06-18/47b431de-f9d5-4dfd-8dba-98f2a7ba8600.parquet,47b431de-f9d5-4dfd-8dba-98f2a7ba8600.parquet,9738,1718735079000


## 1. Preparation

#### 1.1 Spark preparation

In [ ]:
# import necessary libaries
from pyspark.sql.functions import *

In [ ]:
# Creating Spark Session
# from pyspark.sql import SparkSession

# spark = (SparkSession
#          .builder
#          .appName("ML Model")
#          .getOrCreate())

# sc = spark.sparkContext

#### 1.2 Prepare a UDF (User Defined Function)
We need to create UDF (user data functions) to embed the ML model we trained in the previous workshop. This model will be used for Posts data sentiment analysis.

User Defined Function: prepare_data(df) Normalize, clean and set case to lowercase for model. Add additional columns to dataframe for reporting purposes: ("business_id","cool","funny","review_id","text","useful","user_id","date")

User Defined Function: predictions_udf(df.ml_model,stringindexer): Loading dataframe, model and stringindexer (decoded labels).

Decoding the indexer from pyspark.ml.feature import StringIndexerModel, IndexToString

In [ ]:
from pyspark.sql.functions import col, regexp_replace, lower, trim
from pyspark.ml import PipelineModel
from pyspark.ml.feature import StringIndexerModel, IndexToString

def prepare_data(df):
    #  Add additional columns to dataframe for reporting purposes
    df = df.select("business_id","cool","funny","review_id","text","useful","user_id","date")
    cleaned = df.withColumn("text", lower(df.text)) \
        .withColumn("text", regexp_replace("text", "[^a-zA-Z\\s]", "")) \
        .withColumn("text", regexp_replace("text", "\\s+", " ")) 
    return cleaned


def predictions_udf(df, ml_model, stringindexer):

    # Load in the saved pipeline model
    model = PipelineModel.load(ml_model)

    # Making the predictions
    prediction = model.transform(df)
 
    # Add additional columns to dataframe for reporting purposes
    predicted = prediction.select(col('business_id'), col('cool'),col('funny'),col('review_id'),col('text'), col('useful'), col('user_id'), col('date'), col('prediction'))

    # Decoding the indexer
    #from pyspark.ml.feature import StringIndexerModel, IndexToString

    # Load in the StringIndexer that was saved
    indexer = StringIndexerModel.load(stringindexer)

    # Initialize the IndexToString converter
    i2s = IndexToString(inputCol = 'prediction', outputCol = 'decoded', labels = indexer.labels)
    converted = i2s.transform(predicted)

    # Display the important columns
    return converted

#### 1.3 Load Posts files and ML model

If you can remember our last workshop for machine learning model training, our trained model was saved to `/mnt/deBDProject/model`. Yours name might be different.  Let's load the Posts files and then ml model. Set model, indexer, reviews file paths

In [ ]:
#Original Statement
#display(dbutils.fs.ls("abfss://de-yelp-train@wcddestorageexternal.dfs.core.windows.net/model/"))

path,name,size,modificationTime
abfss://de-yelp-train@wcddestorageexternal.dfs.core.windows.net/model/metadata/,metadata/,0,1718065696000
abfss://de-yelp-train@wcddestorageexternal.dfs.core.windows.net/model/stages/,stages/,0,1718065697000


Let's load the Posts files and the ml model

In [ ]:
#reviews = spark.read.parquet(path)
#ml_model = "abfss://de-yelp-train@wcddestorageexternal.dfs.core.windows.net/model/"
#stringindexer = "abfss://de-yelp-train@wcddestorageexternal.dfs.core.windows.net/stringindexer"

In [ ]:
reviews = spark.read.parquet(path)
#ml_model = "abfss://de-yelp-train@wcddestorageexternal.dfs.core.windows.net/model/"
#stringindexer = "abfss://de-yelp-train@wcddestorageexternal.dfs.core.windows.net/stringindexer"

ml_model = "/mnt/deBDproject/de-yelp-train/model/"
stringindexer = "/mnt/deBDproject/de-yelp-train/stringindexer"

In [ ]:
display(reviews)

business_id,cool,funny,review_id,text,useful,user_id,date
G_zuuYzVYI6_Yw34cdq_NA,2,8,KCDme8YF5O221g4rTIBdKw,"the following review contains 0% exaggeration: i once waited 40 minutes for a cup of coffee. this place sucks. the decor is frigid and weird, they have weird things going on (they try to sell you all kinds of stuff like used cd's), they are obviously really self-important because everything has a really special ""grooveground"" label on it.... i just plain hate this place. and their service is so slow it's funny. they only have one person working at a time and this poor sucker has to make all this crazy stuff by himself. i've been there about 3 times and left pissed-off in each instance. their menu is ridiculous (really expensive desserts, mocktails, souffles) and everything is overpriced. also, they do not have soymilk (i believe it's on the menu, but they don't have any). i am never going to grooveground again even though it's within 4 blocks of my home.",6,X5EAMkKCHTpmso9ILIFRUg,2024-06-18 18:20:25
KMF17foSqysDXLZKpiPj9A,1,0,7RP8YhlhQ7g0I19i3YAQgQ,"One fo the best places to get your dance on in my opinion. Cover can range from free-$10. So where do I start? There are essentially 5 levels to this place. Dancing goes on all 4 but the most happening spots are on the 1st and 4th to get your groove on. The 2nd level is more like a mezzanine where you can gaze down on all the pple getting down. 3rd floor is for VIP and you can look up through the part of the clear dance floor on the 4th level (ladies beware unless you're wanting to show off your goods). 4th level has a much larger dance area, but is ALWAYS still packed! 5th level is the rooftop and a great escape from the crowds or to catch some fresh air. Great view of the strip too. Usually has a live band playing more mellow music. Update: Wah! Guess I'm a ghetto girl since its one of my more preferred places to do some booty dancing",3,TDXV6AC5PYOZEyN9eeODfA,2024-06-18 18:24:38
KbSpEQ_54ovgNxOoxKP79g,1,4,YRA3s4bWz_VyirszGvLpvA,"this place stinks like medicine. i don't know what it is, but it has smelled that way for YEARS. what the fuck is it?! this is the most highbrow grocery store in the area. like total fance ""gourmet"" foods. but they have a few good things going for them: deli- you can make a sandwich that is pretty fucking fly pizza shop- they make a good slice cokes- good cokes from the fountain in a wax cup, none of that bullshit throw-away plastic novelty shit coffee- pretty rad coffee for good price hot sauce- has one of the better selections of hot sauces in town.. mostly gimmick stuff though fruit- way cheaper than food co-op for a banana still, the stink ruins it. that and the total foofery.",3,CXCvnCfAvkLpFRiqsp26gw,2024-06-18 18:20:25
aJV-u_8zf5vVIaHy7Hs8qw,2,0,64ypOhlV9Nu3WYwSNXAJAg,"Not sure about the previous review's ""vegetarian"" statement, but Chloe is definitely delicious. It's a cute little BYOB. We had the special appetizer, an amazing gnocchi dish with a meat sauce (lamb, beef, and something else?) that exploded with flavor. The halibut was good and plentiful, served on delicious heirloom tomatoes with micro-arugala. Our other dish was goat cheese/ricotta ravioli with edamame, toasted, pine nuts, and mushrooms. It was a rich dish. Dessert was a berry cobbler. The berries were yummy, but the vanilla ice cream stole the show! I can't believe I forgot to ask if they made it or if I could buy that brand somewhere!",5,gfQqQYI5_hCAGEHlHXIz2Q,2024-06-18 18:20:25
KC8_Rx4Orlsz8LIonCYXsA,1,0,MOZ4DMaS5ZqxZ9IQ9Lo9lg,"Every time I go up to Santa Barbara to go walking on State Street (and no, not as in ""street-walking"" lol...more like shop-perusing!)...I make a stop at the Santa Barbara Brewing Company. During the fall especially, since there seems to be so many more sporting events on TV...this place rocks. Plenty of BIG screens and TV's showing a vast array of stuff for your viewing pleasure. They bre their own beer on site...the Blonde, Hef, IPA, Red

#### 1.4 Run model to predict lables

In [ ]:
# # Producing lables
cleaned = prepare_data(reviews)
result = predictions_udf(cleaned,ml_model, stringindexer)
display(result)

text,prediction,decoded
the following review contains exaggeration i once waited minutes for a cup of coffee this place sucks the decor is frigid and weird they have weird things going on they try to sell you all kinds of stuff like used cds they are obviously really selfimportant because everything has a really special grooveground label on it i just plain hate this place and their service is so slow its funny they only have one person working at a time and this poor sucker has to make all this crazy stuff by himself ive been there about times and left pissedoff in each instance their menu is ridiculous really expensive desserts mocktails souffles and everything is overpriced also they do not have soymilk i believe its on the menu but they dont have any i am never going to grooveground again even though its within blocks of my home,2.0,1.0
one fo the best places to get your dance on in my opinion cover can range from free so where do i start there are essentially levels to this place dancing goes on all but the most happening spots are on the st and th to get your groove on the nd level is more like a mezzanine where you can gaze down on all the pple getting down rd floor is for vip and you can look up through the part of the clear dance floor on the th level ladies beware unless youre wanting to show off your goods th level has a much larger dance area but is always still packed th level is the rooftop and a great escape from the crowds or to catch some fresh air great view of the strip too usually has a live band playing more mellow music update wah guess im a ghetto girl since its one of my more preferred places to do some booty dancing,3.0,3.0
this place stinks like medicine i dont know what it is but it has smelled that way for years what the fuck is it this is the most highbrow grocery store in the area like total fance gourmet foods but they have a few good things going for them deli you can make a sandwich that is pretty fucking fly pizza shop they make a good slice cokes good cokes from the fountain in a wax cup none of that bullshit throwaway plastic novelty shit coffee pretty rad coffee for good price hot sauce has one of the better selections of hot sauces in town mostly gimmick stuff though fruit way cheaper than food coop for a banana still the stink ruins it that and the total foofery,3.0,3.0
not sure about the previous reviews vegetarian statement but chloe is definitely delicious its a cute little byob we had the special appetizer an amazing gnocchi dish with a meat sauce lamb beef and something else that exploded with flavor the halibut was good and plentiful served on delicious heirloom tomatoes with microarugala our other dish was goat cheesericotta ravioli with edamame toasted pine nuts and mushrooms it was a rich dish dessert was a berry cobbler the berries were yummy but the vanilla ice cream stole the show i cant believe i forgot to ask if they made it or if i could buy that brand somewhere,0.0,5.0
every time i go up to santa barbara to go walking on state street and no not as in streetwalking lolmore like shopperusingi make a stop at the santa barbara brewing company during the fall especially since there seems to be so many more sporting events on tvthis place rocks plenty of big screens and tvs showing a vast array of stuff for your viewing pleasure they bre their own beer on sitethe blonde hef ipa red and stout are their poular onesand you can get a little sampler of if you cant make up your mind definitely order yourself a side of their garlic and rosemary fries with the side of ranchyou and your tummy will thank me later your loved ones however might not,0.0,5.0
so i remember when this place used to be a beautiful restaurant even if the food wasnt always teh greatest then they turned the main entry into a sam ash sniff luckily the main portion of the place has been converted into a special events venue for weddings private parties and proms the gardens are very beautiful and from the looks of it 

#### 1.5 Summarize which topics are the most popular

Commented out this step that limits data in the predictions dataframe and aggregates data grouped by stars column. Need all columns and detailed data for reporting purposes in Pow r .


In [ ]:
# change the column name 
#predictions = result.withColumnRenamed('decoded', 'stars').select('stars')

# Aggregate the topics and calculate the total qty of each review level
#predictions_qty = topics.groupBy(col("stars")).agg(count('stars').alias('qty')).orderBy(desc('qty'))
#predictions_qty.show()

+-----+---+
|stars|qty|
+-----+---+
|  5.0| 13|
|  4.0|  5|
|  1.0|  2|
|  3.0|  2|
|  2.0|  1|
+-----+---+



#### 1.6 Save the result file to the `BI` folder


Since spark is a distribution system, if you don't anything, the file you saved will be a folder with a couple of files. The files in the folder will be like this.
<img src='https://s3.amazonaws.com/weclouddata/images/data_engineer/ml_prd2.jpg' width='40%'>

Feel free to add logic to crt_sgl_file function to save just csv file

In [ ]:
# Define this function to save the result DataFrame as a single CSV file
#def crt_sgl_file(result_path, df):
    # Repartition df to a single file and write it directly to the result path as a CSV file
    #df.coalesce(1).write.mode("overwrite").option("header", "true").csv(result_path)

    #print('Single CSV file created')

# Example usage:
# save_as_single_csv("abfss://de-yelp-train@wcddestorageexternal.dfs.core.windows.net/results/", predictions_qty)


In [ ]:
# Define this function to save the result DataFrame as a single CSV file
def crt_sgl_file(result_path, df):
    # Repartition df to a single file and write it directly to the result path as a CSV file
    #df.coalesce(1).write.mode("overwrite").option("header", "true").csv(result_path)
    df.coalesce(1).write.mode("overwrite").option("header", "true").parquet(result_path)

    print('Single Parquet file created')

# Example usage:
# save_as_single_csv("abfss://de-yelp-train@wcddestorageexternal.dfs.core.windows.net/results/", predictions_qty)

In [ ]:
# run the function

crt_sgl_file("abfss://de-yelp-train@wcddestorageexternal.dfs.core.windows.net/results/", predictions_qty)

Single CSV file created
